In [1]:
import os 
import json 
relative_path = '../data/askextension/2020-08-20/'
paths = [f"{relative_path}{fn}"for fn in os.listdir(relative_path)]

In [2]:
paths

['../data/askextension/2020-08-20/2012-2014.json',
 '../data/askextension/2020-08-20/2014-2016.json',
 '../data/askextension/2020-08-20/2016-2018.json',
 '../data/askextension/2020-08-20/2018-2020.json']

In [3]:
## Viewing format:

with open(paths[0], 'r', encoding='utf-8') as r:
    sample = json.load(r)[0]
sample

{'faq-id': 1,
 'title': 'When can I plant blue spruce trees in Colorado? #109900',
 'created': '2012-12-03 15:53:47',
 'updated': '2012-12-03 17:47:21',
 'tags': ['trees and shrubs'],
 'state': 'Colorado',
 'county': 'El Paso County',
 'question': 'I need to plant two blue spruce trees that are currently in 24" diameter plastic containers with drain holes in the bottom sides.\n\nLocation: northeast side of Colorado Springs.\n\nThese trees are currently outside on the patio and susceptible to the wind and sun. The trees were watered this past Saturday and seem to be healthy.\n\nQuestion: Can these trees be planted now? Currently the soil is not frozen and night time temps are 35 to 40 degrees.\n\nI have downloaded and read CMG GardenNotes #633 as a reference.\n\nAny advice would be greatly appreciated. ',
 'answer': {'1': {'response': 'Jerry, \nyou can plant them now (a) OR temporarily "plant" them, still in containers, so that roots have some insulation from cold (b).\n\n(a) if you kno

In [4]:
from elasticsearch import Elasticsearch
url = 'http://localhost:9200'
url = 'https://dev.es.chat.ask.eduworks.com/'
#url = 'https://qa.es.chat.ask.eduworks.com/'
es = Elasticsearch([url], http_auth=('elastic', 'changeme'))

In [5]:
es.indices.get_mapping(index='combined')

{'combined': {'mappings': {'dynamic': 'false',
   'properties': {'damage': {'type': 'keyword',
     'index': False,
     'ignore_above': 32766},
    'description': {'type': 'keyword', 'index': False, 'ignore_above': 32766},
    'development': {'type': 'keyword', 'index': False, 'ignore_above': 32766},
    'identification': {'type': 'keyword',
     'index': False,
     'ignore_above': 32766},
    'links': {'type': 'nested',
     'properties': {'link': {'type': 'keyword',
       'index': False,
       'ignore_above': 32766},
      'src': {'type': 'keyword', 'index': False, 'ignore_above': 32766},
      'title': {'type': 'keyword', 'index': False, 'ignore_above': 32766},
      'type': {'type': 'keyword', 'index': False, 'ignore_above': 32766}}},
    'management': {'type': 'keyword', 'index': False, 'ignore_above': 32766},
    'source': {'type': 'keyword', 'ignore_above': 32766},
    'title': {'type': 'keyword', 'index': False, 'ignore_above': 32766},
    'url': {'type': 'keyword', 'index'

In [6]:
es.cat.indices(h='index', s='index').split()

['.geoip_databases', 'combined', 'logs']

In [7]:
from tqdm import tqdm
from datetime import datetime

def ingest_ask_data():
    for path in tqdm(paths): 
        with open(path, 'r', encoding='utf-8') as r:
            kb_data = json.load(r)
        for item in tqdm(kb_data): 
            item['created_timestamp'] = datetime.strptime(item['created'].split()[0], '%Y-%m-%d').timestamp()
            item['updated_timestamp'] = datetime.strptime(item['created'].split()[0], '%Y-%m-%d').timestamp()
            res = es.index(index='ask_extension_kb', body=item)
ingest_ask_data()

  0%|          | 0/4 [00:00<?, ?it/s]C:\Users\imjef\AppData\Local\Temp\ipykernel_5680\4065793934.py:11: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.index(index='ask_extension_kb', body=item)
 75%|███████▌  | 3/4 [1:24:24<31:36, 1896.79s/it]

In [ ]:
es.indices.get_mapping(index='ask_extension_kb')

In [ ]:
query = {
    'aggs': {
        'last_updated':{
            'max': {
                'field': 'created_timestamp'
            }
        }
    }
}

In [ ]:
es.search(index='ask_extension_kb', body=query, size=0)['aggregations']['last_updated']['value']